In [1]:
# Dependencies
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import Input
from tensorflow.keras.callbacks import EarlyStopping

# Load data (FRED-MD)
df = pd.read_csv('../current.csv')

# Remove the first row (transformation codes)
transformation_codes = df.iloc[0]  # Transformation codes can be applied if needed
df = df.iloc[1:]

# Set the first column as the index and datetime
df.set_index(df.columns[0], inplace=True)
df.index = pd.to_datetime(df.index)

# Dropna
data = df.dropna()

# Create train data and target
target = (data['CPIAUCSL'].diff(12) / data['CPIAUCSL'].shift(12)) * 100
target = target.shift(-12).dropna()
data = data.loc[target.index]
train = data.dropna()

2025-04-06 08:04:48.278969: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 08:04:48.280100: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 08:04:48.289532: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 08:04:48.303977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743941088.324417  358787 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743941088.33

In [2]:
horizons = list(range(4, 12*5 + 4, 12))
for forecast_horizon in horizons:
    series = target.values  # our univariate series (change in CPI)

    for context_window in [1, 2, 4, 6, 12, 24]:
        for dropout_rate in [0.0, 0.1, 0.2, 0.3]:
            # how many training windows we can make before the final test block
            cutoff = len(series) - forecast_horizon

            # — build X (context_window steps) and y (next step) —
            X, y = [], []
            for i in range(cutoff - context_window):
                X.append(series[i : i + context_window])
                y.append(series[i + context_window])
            X = np.array(X).reshape(-1, context_window, 1)
            y = np.array(y).reshape(-1, 1)

            # — scale inputs and outputs separately —
            scaler_X = MinMaxScaler()
            X_scaled = scaler_X.fit_transform(
                X.reshape(-1, 1)
            ).reshape(X.shape)

            scaler_y = MinMaxScaler()
            y_scaled = scaler_y.fit_transform(y)

            # — train / val / test split —
            n = X_scaled.shape[0]
            test_size = forecast_horizon
            val_size = int((n - test_size) * 0.1)

            X_train = X_scaled[: n - test_size - val_size]
            y_train = y_scaled[: n - test_size - val_size]
            X_val   = X_scaled[n - test_size - val_size : n - test_size]
            y_val   = y_scaled[n - test_size - val_size : n - test_size]

            # — build a one‐step LSTM —
            model = Sequential([
                Input((context_window, 1)),
                LSTM(64, return_sequences=True),
                Dropout(dropout_rate),
                LSTM(64),
                Dropout(dropout_rate),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse')

            # — fit with early stopping —
            es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            model.fit(
                X_train, y_train,
                epochs=100,
                batch_size=32,
                shuffle=False,
                validation_data=(X_val, y_val),
                callbacks=[es],
                verbose=0
            )

            # — iterative, autoregressive forecasting —
            history = series[-context_window:].copy()
            forecast = []
            for _ in range(forecast_horizon):
                x_in = scaler_X.transform(history.reshape(-1,1)).reshape(1, context_window, 1)
                yhat_s = model.predict(x_in, verbose=0)
                yhat   = scaler_y.inverse_transform(yhat_s)[0,0]
                forecast.append(yhat)
                # roll the window forward
                history = np.append(history[1:], yhat)

            # — compute metrics on the last `forecast_horizon` true vals —
            y_true = series[-forecast_horizon:]
            rmse = np.sqrt(mean_squared_error(y_true, forecast))
            mae  = mean_absolute_error(y_true, forecast)

            # — log to CSV  —
            fp = "uni_ar_lstm.csv"
            df_log = pd.read_csv(fp)
            new_row = {
                'forecast_horizon': forecast_horizon,
                'context_window':   context_window,
                'dropout_rate':     dropout_rate,
                'rmse':             rmse,
                'mae':              mae,
                'forecast':         list(forecast),
                'true_vals':        list(y_true.tolist())
            }
            df_log = pd.concat([df_log, pd.DataFrame([new_row])], ignore_index=True)
            df_log.to_csv(fp, index=False)


2025-04-06 08:08:52.629326: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/tmp/ipykernel_358787/2541705606.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_log = pd.concat([df_log, pd.DataFrame([new_row])], ignore_index=True)
